# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff
using PseudoPotentialData

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice = T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
    atoms     = [ElementPsp(:He, pseudopotentials)]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.771002743383                   -0.53    9.0    176ms
  2   -2.772151607798       -2.94       -1.32    1.0    105ms
  3   -2.772170426366       -4.73       -2.50    1.0    101ms
  4   -2.772170667643       -6.62       -3.26    1.0    145ms
  5   -2.772170722171       -7.26       -4.01    2.0    445ms
  6   -2.772170722851       -9.17       -4.44    1.0    103ms
  7   -2.772170723010       -9.80       -5.32    1.0    135ms
  8   -2.772170723015      -11.30       -6.50    2.0    132ms
  9   -2.772170723015   +  -14.24       -6.49    2.0    119ms
 10   -2.772170723015      -13.48       -7.27    1.0    108ms
 11   -2.772170723015   +  -14.01       -7.69    1.0    114ms
 12   -2.772170723015      -14.07       -8.19    2.0    125ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.773557946792601

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770813722049                   -0.52    8.0    160ms
  2   -2.772059651235       -2.90       -1.32    1.0    106ms
  3   -2.772083166579       -4.63       -2.51    1.0    100ms
  4   -2.772083366651       -6.70       -3.28    1.0    102ms
  5   -2.772083415890       -7.31       -3.88    2.0    118ms
  6   -2.772083417778       -8.72       -5.18    1.0    485ms
  7   -2.772083417810      -10.50       -5.45    2.0    116ms
  8   -2.772083417811      -12.14       -6.51    1.0    105ms
  9   -2.772083417811      -13.67       -7.20    2.0    118ms
 10   -2.772083417811      -15.05       -7.77    1.0    107ms
 11   -2.772083417811      -14.40       -8.64    1.0    107ms
Solving response problem
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0